In [2]:
# Download real AFM images
!wget -q https://zenodo.org/api/records/60434/files-archive -O afm_data.zip
!unzip -q afm_data.zip

# List downloaded files
import os
print("Downloaded AFM files:")
for root, dirs, files in os.walk("."):
    for file in files:
        if file.endswith(('.png', '.mi', '.tif', '.tiff')):
            print(os.path.join(root, file))

Downloaded AFM files:
./image_7.mi
./image_12.png
./image_16.mi
./image_13.mi
./image_16.png
./image_13.png
./image_14.mi
./image_8.png
./image_10.mi
./image_7.png
./image_10.png
./image_11.png
./image_8.mi
./image_15.png
./image_11.mi
./image_15.mi
./image_14.png
./image_12.mi
./image_9.png
./image_9.mi


In [3]:
# Install required packages for Google Colab
try:
    import google.colab
    IN_COLAB = True
    # Install missing packages
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "reportlab", "scikit-image"])
except:
    IN_COLAB = False

import gradio as gr
import numpy as np
from PIL import Image
import io
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
from scipy import ndimage, signal
from scipy.optimize import curve_fit
from skimage import filters, morphology, measure
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image as RLImage, Table, TableStyle, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.lib.enums import TA_CENTER
import tempfile
import os

# Cell type compatibility parameters (from literature)
CELL_PARAMETERS = {
    "Osteoblasts (MG-63)": {
        "roughness_range": (10, 500),  # nm, optimal range
        "roughness_optimal": (50, 200),  # nm, best range
        "modulus_range": (1, 100),  # kPa, acceptable range
        "modulus_optimal": (10, 50),  # kPa, best range
        "adhesion_range": (50, 5000),  # pN, typical range
        "adhesion_optimal": (200, 2000),  # pN, best for proliferation
        "response": "Increased proliferation with roughness",
        "color": "#FF6B6B"
    },
    "Fibroblasts (L929/HDF)": {
        "roughness_range": (1, 100),  # nm, optimal range
        "roughness_optimal": (2, 50),  # nm, best range
        "modulus_range": (0.5, 50),  # kPa
        "modulus_optimal": (1, 20),  # kPa
        "adhesion_range": (20, 2000),  # pN
        "adhesion_optimal": (50, 800),  # pN
        "response": "Decreased proliferation with high roughness",
        "color": "#4ECDC4"
    },
    "HeLa Cells": {
        "roughness_range": (5, 150),  # nm
        "roughness_optimal": (10, 80),  # nm
        "modulus_range": (0.5, 30),  # kPa
        "modulus_optimal": (2, 15),  # kPa
        "adhesion_range": (30, 1500),  # pN
        "adhesion_optimal": (100, 800),  # pN
        "response": "Moderate roughness preference",
        "color": "#95E1D3"
    },
    "Endothelial Cells": {
        "roughness_range": (5, 100),  # nm
        "roughness_optimal": (10, 60),  # nm
        "modulus_range": (0.5, 25),  # kPa
        "modulus_optimal": (1, 10),  # kPa
        "adhesion_range": (50, 1800),  # pN
        "adhesion_optimal": (150, 1000),  # pN
        "response": "Smooth to moderate roughness",
        "color": "#F38181"
    }
}

def analyze_afm_image(image_array):
    """Extract surface metrics from AFM image"""
    # Ensure grayscale
    if len(image_array.shape) == 3:
        image_array = np.mean(image_array, axis=2)

    # Normalize to nm scale (assuming typical AFM height range)
    height_map = (image_array / 255.0) * 1000  # 0-1000 nm range

    # Denoise with Gaussian filter
    denoised = filters.gaussian(height_map, sigma=1.5)

    # Calculate RMS roughness
    mean_height = np.mean(denoised)
    rms_roughness = np.sqrt(np.mean((denoised - mean_height)**2))

    # Calculate Ra (average roughness)
    ra_roughness = np.mean(np.abs(denoised - mean_height))

    # Calculate peak-to-valley
    pv_roughness = np.max(denoised) - np.min(denoised)

    # Extract gradient features for stiffness estimation
    gy, gx = np.gradient(denoised)
    gradient_magnitude = np.sqrt(gx**2 + gy**2)
    avg_gradient = np.mean(gradient_magnitude)

    # Estimate Young's modulus from surface features
    # Higher gradients and roughness correlate with stiffer surfaces
    estimated_modulus = 5 + (avg_gradient * 0.5) + (rms_roughness * 0.1)
    estimated_modulus = np.clip(estimated_modulus, 0.5, 150)

    # Estimate adhesion force based on surface characteristics
    # Rougher surfaces typically show higher adhesion forces
    estimated_adhesion = 100 + (rms_roughness * 10) + (avg_gradient * 50)
    estimated_adhesion = np.clip(estimated_adhesion, 20, 5000)

    # Calculate surface area ratio
    pixel_size = 1.0  # nm per pixel (adjustable)
    surface_area = np.sum(np.sqrt(1 + gx**2 + gy**2)) * pixel_size**2
    projected_area = denoised.size * pixel_size**2
    surface_area_ratio = surface_area / projected_area

    # Frequency analysis
    fft = np.fft.fft2(denoised)
    psd = np.abs(fft)**2
    avg_spatial_freq = np.mean(psd[1:10, 1:10])

    metrics = {
        "rms_roughness": rms_roughness,
        "ra_roughness": ra_roughness,
        "pv_roughness": pv_roughness,
        "modulus": estimated_modulus,
        "adhesion": estimated_adhesion,
        "gradient": avg_gradient,
        "surface_area_ratio": surface_area_ratio,
        "spatial_frequency": avg_spatial_freq,
        "height_map": denoised
    }

    return metrics

def calculate_compatibility_score(metrics, cell_type):
    """Calculate compatibility score for specific cell type"""
    params = CELL_PARAMETERS[cell_type]

    # Score roughness (40% weight)
    rough = metrics["rms_roughness"]
    if params["roughness_optimal"][0] <= rough <= params["roughness_optimal"][1]:
        rough_score = 100
    elif params["roughness_range"][0] <= rough <= params["roughness_range"][1]:
        # Linear decay outside optimal range
        if rough < params["roughness_optimal"][0]:
            rough_score = 50 + 50 * (rough - params["roughness_range"][0]) / (params["roughness_optimal"][0] - params["roughness_range"][0])
        else:
            rough_score = 50 + 50 * (params["roughness_range"][1] - rough) / (params["roughness_range"][1] - params["roughness_optimal"][1])
    else:
        rough_score = max(0, 30 - abs(rough - np.mean(params["roughness_optimal"])) * 0.5)

    # Score modulus (35% weight)
    mod = metrics["modulus"]
    if params["modulus_optimal"][0] <= mod <= params["modulus_optimal"][1]:
        mod_score = 100
    elif params["modulus_range"][0] <= mod <= params["modulus_range"][1]:
        if mod < params["modulus_optimal"][0]:
            mod_score = 50 + 50 * (mod - params["modulus_range"][0]) / (params["modulus_optimal"][0] - params["modulus_range"][0])
        else:
            mod_score = 50 + 50 * (params["modulus_range"][1] - mod) / (params["modulus_range"][1] - params["modulus_optimal"][1])
    else:
        mod_score = max(0, 30 - abs(mod - np.mean(params["modulus_optimal"])) * 0.5)

    # Score adhesion (25% weight)
    adh = metrics["adhesion"]
    if params["adhesion_optimal"][0] <= adh <= params["adhesion_optimal"][1]:
        adh_score = 100
    elif params["adhesion_range"][0] <= adh <= params["adhesion_range"][1]:
        if adh < params["adhesion_optimal"][0]:
            adh_score = 50 + 50 * (adh - params["adhesion_range"][0]) / (params["adhesion_optimal"][0] - params["adhesion_range"][0])
        else:
            adh_score = 50 + 50 * (params["adhesion_range"][1] - adh) / (params["adhesion_range"][1] - params["adhesion_optimal"][1])
    else:
        adh_score = max(0, 30 - abs(adh - np.mean(params["adhesion_optimal"])) * 0.02)

    # Weighted total score
    total_score = (rough_score * 0.40 + mod_score * 0.35 + adh_score * 0.25)

    # Determine pass/fail and proliferation potential
    if total_score >= 75:
        status = "EXCELLENT"
        proliferation = "High proliferation expected"
    elif total_score >= 60:
        status = "GOOD"
        proliferation = "Moderate-to-high proliferation"
    elif total_score >= 45:
        status = "ACCEPTABLE"
        proliferation = "Moderate proliferation"
    else:
        status = "POOR"
        proliferation = "Low proliferation, not recommended"

    return {
        "total_score": total_score,
        "rough_score": rough_score,
        "mod_score": mod_score,
        "adh_score": adh_score,
        "status": status,
        "proliferation": proliferation
    }

def create_visualizations(metrics, cell_type, scores):
    """Create analysis visualizations"""
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f'AFM Biomaterial Analysis - {cell_type}', fontsize=16, fontweight='bold')

    # 1. Height map
    im1 = axes[0, 0].imshow(metrics["height_map"], cmap='viridis')
    axes[0, 0].set_title('Surface Topography')
    axes[0, 0].set_xlabel('X position')
    axes[0, 0].set_ylabel('Y position')
    plt.colorbar(im1, ax=axes[0, 0], label='Height (nm)')

    # 2. Height histogram
    axes[0, 1].hist(metrics["height_map"].flatten(), bins=50, color='steelblue', alpha=0.7)
    axes[0, 1].set_title('Height Distribution')
    axes[0, 1].set_xlabel('Height (nm)')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].axvline(np.mean(metrics["height_map"]), color='red', linestyle='--', label='Mean')
    axes[0, 1].legend()

    # 3. Gradient/roughness map
    gy, gx = np.gradient(metrics["height_map"])
    gradient_mag = np.sqrt(gx**2 + gy**2)
    im3 = axes[0, 2].imshow(gradient_mag, cmap='hot')
    axes[0, 2].set_title('Surface Gradient (Roughness)')
    axes[0, 2].set_xlabel('X position')
    axes[0, 2].set_ylabel('Y position')
    plt.colorbar(im3, ax=axes[0, 2], label='Gradient')

    # 4. Score breakdown
    categories = ['Roughness\n(40%)', 'Modulus\n(35%)', 'Adhesion\n(25%)']
    component_scores = [scores["rough_score"], scores["mod_score"], scores["adh_score"]]
    colors_bar = ['#FF6B6B', '#4ECDC4', '#95E1D3']
    bars = axes[1, 0].bar(categories, component_scores, color=colors_bar, alpha=0.7)
    axes[1, 0].set_ylim(0, 100)
    axes[1, 0].set_ylabel('Score')
    axes[1, 0].set_title('Compatibility Component Scores')
    axes[1, 0].axhline(75, color='green', linestyle='--', label='Excellent', linewidth=1)
    axes[1, 0].axhline(60, color='orange', linestyle='--', label='Good', linewidth=1)
    axes[1, 0].axhline(45, color='red', linestyle='--', label='Acceptable', linewidth=1)
    for bar, score in zip(bars, component_scores):
        height = bar.get_height()
        axes[1, 0].text(bar.get_x() + bar.get_width()/2., height,
                       f'{score:.1f}', ha='center', va='bottom', fontweight='bold')
    axes[1, 0].legend(loc='upper right', fontsize=8)

    # 5. Overall compatibility gauge
    ax_gauge = axes[1, 1]
    ax_gauge.set_xlim(0, 10)
    ax_gauge.set_ylim(0, 10)
    ax_gauge.axis('off')

    # Draw gauge
    theta = np.linspace(np.pi, 0, 100)
    r = 4
    x_arc = 5 + r * np.cos(theta)
    y_arc = 3 + r * np.sin(theta)
    ax_gauge.plot(x_arc, y_arc, 'k-', linewidth=3)

    # Color zones
    score_angle = np.pi * (1 - scores["total_score"] / 100)
    ax_gauge.fill_between([5 + r * np.cos(t) for t in np.linspace(np.pi, 2*np.pi/3, 30)],
                          [3 + r * np.sin(t) for t in np.linspace(np.pi, 2*np.pi/3, 30)],
                          3, color='red', alpha=0.2)
    ax_gauge.fill_between([5 + r * np.cos(t) for t in np.linspace(2*np.pi/3, np.pi/3, 30)],
                          [3 + r * np.sin(t) for t in np.linspace(2*np.pi/3, np.pi/3, 30)],
                          3, color='yellow', alpha=0.2)
    ax_gauge.fill_between([5 + r * np.cos(t) for t in np.linspace(np.pi/3, 0, 30)],
                          [3 + r * np.sin(t) for t in np.linspace(np.pi/3, 0, 30)],
                          3, color='green', alpha=0.2)

    # Score needle
    needle_x = 5 + (r-0.5) * np.cos(score_angle)
    needle_y = 3 + (r-0.5) * np.sin(score_angle)
    ax_gauge.plot([5, needle_x], [3, needle_y], 'r-', linewidth=4)
    ax_gauge.plot(5, 3, 'ko', markersize=10)

    # Score text
    ax_gauge.text(5, 1, f'{scores["total_score"]:.1f}%',
                 ha='center', va='top', fontsize=24, fontweight='bold')
    ax_gauge.text(5, 0.3, scores["status"],
                 ha='center', va='top', fontsize=14, fontweight='bold',
                 color='green' if scores["total_score"] >= 75 else 'orange' if scores["total_score"] >= 60 else 'red')
    ax_gauge.set_title('Overall Compatibility Score', fontsize=12, fontweight='bold')

    # 6. Comparison with cell type ranges
    ax_ranges = axes[1, 2]
    params = CELL_PARAMETERS[cell_type]

    # Roughness comparison
    y_pos = [2.5, 1.5, 0.5]
    labels = ['Roughness (nm)', 'Modulus (kPa)', 'Adhesion (pN)']
    optimal_ranges = [params["roughness_optimal"], params["modulus_optimal"], params["adhesion_optimal"]]
    acceptable_ranges = [params["roughness_range"], params["modulus_range"], params["adhesion_range"]]
    measured_values = [metrics["rms_roughness"], metrics["modulus"], metrics["adhesion"]]

    for i, (y, label, opt, acc, val) in enumerate(zip(y_pos, labels, optimal_ranges, acceptable_ranges, measured_values)):
        # Acceptable range (light)
        ax_ranges.barh(y, acc[1] - acc[0], left=acc[0], height=0.3,
                      color='lightblue', alpha=0.5, label='Acceptable' if i == 0 else '')
        # Optimal range (dark)
        ax_ranges.barh(y, opt[1] - opt[0], left=opt[0], height=0.3,
                      color='darkblue', alpha=0.7, label='Optimal' if i == 0 else '')
        # Measured value
        ax_ranges.plot(val, y, 'r*', markersize=15, label='Measured' if i == 0 else '')

    ax_ranges.set_yticks(y_pos)
    ax_ranges.set_yticklabels(labels)
    ax_ranges.set_xlabel('Value')
    ax_ranges.set_title('Measured vs. Optimal Ranges')
    ax_ranges.legend(loc='upper right', fontsize=8)
    ax_ranges.grid(axis='x', alpha=0.3)

    plt.tight_layout()

    # Save to bytes
    buf = io.BytesIO()
    plt.savefig(buf, format='png', dpi=150, bbox_inches='tight')
    buf.seek(0)
    plt.close()

    return buf

def generate_pdf_report(metrics, cell_type, scores, viz_buffer):
    """Generate comprehensive PDF report"""
    temp_pdf = tempfile.NamedTemporaryFile(delete=False, suffix='.pdf')
    doc = SimpleDocTemplate(temp_pdf.name, pagesize=letter)

    styles = getSampleStyleSheet()
    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Heading1'],
        fontSize=24,
        textColor=colors.HexColor('#2C3E50'),
        spaceAfter=30,
        alignment=TA_CENTER
    )

    story = []

    # Title
    story.append(Paragraph("AFM Biomaterial Compatibility Analysis Report", title_style))
    story.append(Spacer(1, 0.3*inch))

    # Cell type and overall score
    story.append(Paragraph(f"<b>Target Cell Type:</b> {cell_type}", styles['Heading2']))
    story.append(Spacer(1, 0.2*inch))

    score_color = 'green' if scores["total_score"] >= 75 else 'orange' if scores["total_score"] >= 60 else 'red'
    story.append(Paragraph(f"<b>Overall Compatibility Score: <font color='{score_color}'>{scores['total_score']:.1f}%</font></b>", styles['Heading2']))
    story.append(Paragraph(f"<b>Status:</b> {scores['status']}", styles['Normal']))
    story.append(Paragraph(f"<b>Proliferation Potential:</b> {scores['proliferation']}", styles['Normal']))
    story.append(Spacer(1, 0.3*inch))

    # Measured metrics table
    story.append(Paragraph("<b>Measured Surface Metrics</b>", styles['Heading2']))
    story.append(Spacer(1, 0.1*inch))

    metrics_data = [
        ['Parameter', 'Value', 'Unit'],
        ['RMS Roughness', f"{metrics['rms_roughness']:.2f}", 'nm'],
        ['Ra Roughness', f"{metrics['ra_roughness']:.2f}", 'nm'],
        ['Peak-to-Valley', f"{metrics['pv_roughness']:.2f}", 'nm'],
        ['Young\'s Modulus (est.)', f"{metrics['modulus']:.2f}", 'kPa'],
        ['Adhesion Force (est.)', f"{metrics['adhesion']:.1f}", 'pN'],
        ['Surface Area Ratio', f"{metrics['surface_area_ratio']:.3f}", '-'],
    ]

    metrics_table = Table(metrics_data, colWidths=[3*inch, 1.5*inch, 1*inch])
    metrics_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 12),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))

    story.append(metrics_table)
    story.append(Spacer(1, 0.3*inch))

    # Component scores table
    story.append(Paragraph("<b>Compatibility Component Scores</b>", styles['Heading2']))
    story.append(Spacer(1, 0.1*inch))

    scores_data = [
        ['Component', 'Weight', 'Score', 'Status'],
        ['Surface Roughness', '40%', f"{scores['rough_score']:.1f}%",
         'Optimal' if scores['rough_score'] >= 75 else 'Good' if scores['rough_score'] >= 60 else 'Acceptable' if scores['rough_score'] >= 45 else 'Poor'],
        ['Mechanical Modulus', '35%', f"{scores['mod_score']:.1f}%",
         'Optimal' if scores['mod_score'] >= 75 else 'Good' if scores['mod_score'] >= 60 else 'Acceptable' if scores['mod_score'] >= 45 else 'Poor'],
        ['Adhesion Properties', '25%', f"{scores['adh_score']:.1f}%",
         'Optimal' if scores['adh_score'] >= 75 else 'Good' if scores['adh_score'] >= 60 else 'Acceptable' if scores['adh_score'] >= 45 else 'Poor'],
    ]

    scores_table = Table(scores_data, colWidths=[2*inch, 1*inch, 1.5*inch, 1.5*inch])
    scores_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 12),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.lightblue),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))

    story.append(scores_table)
    story.append(Spacer(1, 0.3*inch))

    # Interpretation
    story.append(Paragraph("<b>Interpretation & Recommendations</b>", styles['Heading2']))
    story.append(Spacer(1, 0.1*inch))

    params = CELL_PARAMETERS[cell_type]
    interpretation = f"""
    <b>Cell Response Profile:</b> {params['response']}<br/><br/>
    <b>Roughness Analysis:</b> The measured RMS roughness of {metrics['rms_roughness']:.1f} nm is
    {'within the optimal range' if params['roughness_optimal'][0] <= metrics['rms_roughness'] <= params['roughness_optimal'][1]
    else 'within the acceptable range' if params['roughness_range'][0] <= metrics['rms_roughness'] <= params['roughness_range'][1]
    else 'outside the recommended range'} for {cell_type}.
    Optimal range: {params['roughness_optimal'][0]}-{params['roughness_optimal'][1]} nm.<br/><br/>
    <b>Mechanical Properties:</b> The estimated Young's modulus of {metrics['modulus']:.1f} kPa suggests
    {'suitable stiffness' if params['modulus_optimal'][0] <= metrics['modulus'] <= params['modulus_optimal'][1]
    else 'acceptable stiffness' if params['modulus_range'][0] <= metrics['modulus'] <= params['modulus_range'][1]
    else 'suboptimal stiffness'} for cell attachment and proliferation via integrin signaling.<br/><br/>
    <b>Adhesion Properties:</b> The estimated adhesion force of {metrics['adhesion']:.0f} pN indicates
    {'strong cell-substrate interaction potential' if params['adhesion_optimal'][0] <= metrics['adhesion'] <= params['adhesion_optimal'][1]
    else 'moderate interaction potential' if params['adhesion_range'][0] <= metrics['adhesion'] <= params['adhesion_range'][1]
    else 'weak interaction potential'}.<br/><br/>
    <b>Overall Recommendation:</b> This biomaterial surface is <b>{scores['status']}</b> for {cell_type} culture.
    {scores['proliferation']}.
    """

    story.append(Paragraph(interpretation, styles['Normal']))
    story.append(PageBreak())

    # Add visualizations
    story.append(Paragraph("<b>Analysis Visualizations</b>", styles['Heading2']))
    story.append(Spacer(1, 0.2*inch))

    # Save viz to temp file
    temp_img = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
    temp_img.write(viz_buffer.getvalue())
    temp_img.close()

    img = RLImage(temp_img.name, width=7*inch, height=4.67*inch)
    story.append(img)

    # Build PDF
    doc.build(story)

    # Clean up temp image
    os.unlink(temp_img.name)

    return temp_pdf.name

def process_afm_image(image, cell_type):
    """Main processing function"""
    if image is None:
        return None, "Please upload an AFM image", None, None

    # Convert PIL image to numpy array
    img_array = np.array(image)

    # Analyze image
    metrics = analyze_afm_image(img_array)

    # Calculate compatibility
    scores = calculate_compatibility_score(metrics, cell_type)

    # Create visualizations
    viz_buffer = create_visualizations(metrics, cell_type, scores)
    viz_img = Image.open(viz_buffer)

    # Generate report
    pdf_path = generate_pdf_report(metrics, cell_type, scores, viz_buffer)

    # Create summary text
    summary = f"""
## Analysis Results for {cell_type}

### Surface Metrics
- **RMS Roughness:** {metrics['rms_roughness']:.2f} nm
- **Ra Roughness:** {metrics['ra_roughness']:.2f} nm
- **Peak-to-Valley:** {metrics['pv_roughness']:.2f} nm
- **Young's Modulus (estimated):** {metrics['modulus']:.2f} kPa
- **Adhesion Force (estimated):** {metrics['adhesion']:.1f} pN
- **Surface Area Ratio:** {metrics['surface_area_ratio']:.3f}

### Compatibility Assessment
- **Overall Score:** {scores['total_score']:.1f}% - **{scores['status']}**
- **Roughness Score:** {scores['rough_score']:.1f}% (40% weight)
- **Modulus Score:** {scores['mod_score']:.1f}% (35% weight)
- **Adhesion Score:** {scores['adh_score']:.1f}% (25% weight)

### Proliferation Potential
{scores['proliferation']}

### Cell Response Profile
{CELL_PARAMETERS[cell_type]['response']}
    """

    return viz_img, summary, pdf_path, None

# Create Gradio interface
with gr.Blocks(title="AFM Biomaterial Compatibility Analyzer", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🔬 AFM Biomaterial Compatibility Analyzer

    Upload AFM or microscope images to analyze surface properties and predict cell compatibility.
    This system extracts roughness, stiffness, and adhesion metrics to assess biomaterial suitability for specific cell types.

    **Based on literature benchmarks for:**
    - Osteoblasts (bone cells): Prefer roughness 10-500 nm, modulus 1-100 kPa
    - Fibroblasts (connective tissue): Prefer smooth surfaces < 100 nm
    - HeLa & Endothelial cells: Moderate roughness preference
    """)

    with gr.Row():
        with gr.Column(scale=1):
            image_input = gr.Image(type="pil", label="Upload AFM/Microscope Image (TIFF, PNG, JPG)")
            cell_type_dropdown = gr.Dropdown(
                choices=list(CELL_PARAMETERS.keys()),
                value="Osteoblasts (MG-63)",
                label="Select Target Cell Type"
            )
            analyze_btn = gr.Button("Analyze Biomaterial", variant="primary", size="lg")

            gr.Markdown("""
            ### 💡 Tips
            - Upload AFM height maps, phase images, or microscopy images
            - Supported formats: TIFF, PNG, JPG
            - Higher resolution images give more accurate results
            - Try generating a test pattern below if you don't have AFM images yet
            """)

            generate_test_btn = gr.Button("Generate Test AFM Surface", size="sm")

            def generate_test_image():
                """Generate a synthetic AFM-like surface for testing"""
                np.random.seed(42)
                size = 512
                # Create base surface with multiple frequency components
                x = np.linspace(0, 10, size)
                y = np.linspace(0, 10, size)
                X, Y = np.meshgrid(x, y)

                # Combine multiple sine waves for realistic topography
                Z = 50 * np.sin(2*X) * np.cos(2*Y)
                Z += 30 * np.sin(5*X + 1) * np.cos(3*Y)
                Z += 20 * np.sin(8*X) * np.cos(6*Y + 2)

                # Add noise for roughness
                Z += np.random.randn(size, size) * 15

                # Add some sharp features (like particles)
                for _ in range(5):
                    cx, cy = np.random.randint(100, 400, 2)
                    r = np.random.randint(20, 50)
                    mask = ((X*size/10 - cx)**2 + (Y*size/10 - cy)**2) < r**2
                    Z[mask] += np.random.randint(50, 150)

                # Normalize to 0-255
                Z = ((Z - Z.min()) / (Z.max() - Z.min()) * 255).astype(np.uint8)

                return Image.fromarray(Z, mode='L')

            generate_test_btn.click(
                fn=generate_test_image,
                outputs=image_input
            )

        with gr.Column(scale=2):
            output_viz = gr.Image(label="Analysis Visualizations")
            output_summary = gr.Markdown(label="Compatibility Summary")

    with gr.Row():
        pdf_output = gr.File(label="Download PDF Report")
        error_output = gr.Textbox(label="Status", visible=False)

    gr.Markdown("""
    ---
    ### How It Works

    1. **Image Preprocessing:** Denoising with Gaussian filters and segmentation
    2. **Metric Extraction:**
       - **RMS Roughness:** Root mean square deviation from mean height
       - **Young's Modulus:** Estimated from surface gradients and topography
       - **Adhesion Force:** Estimated from roughness and surface features
    3. **Compatibility Scoring:** Weighted comparison against cell-type specific benchmarks
       - Roughness: 40% weight (integrin binding sites)
       - Modulus: 35% weight (mechanotransduction)
       - Adhesion: 25% weight (focal adhesion formation)
    4. **Pass/Fail Criteria:**
       - ≥75%: Excellent (high proliferation)
       - ≥60%: Good (moderate-high proliferation)
       - ≥45%: Acceptable (moderate proliferation)
       - <45%: Poor (not recommended)

    ### References
    Analysis based on benchmarks from:
    - BBBC047 Broad Bioimage Benchmark Collection
    - QUAM-AFM quantitative AFM datasets
    - Peer-reviewed studies on cell-biomaterial interactions

    ### Deployment
    - **Host on:** Hugging Face Spaces, GitHub Codespaces
    - **Fork:** [GitHub Repository](#)
    - **Requirements:** `pip install gradio numpy pillow matplotlib scipy scikit-image pandas reportlab`

    💡 **Tip:** For best results, use high-resolution AFM height maps or phase images
    """)

    # Connect the analysis
    analyze_btn.click(
        fn=process_afm_image,
        inputs=[image_input, cell_type_dropdown],
        outputs=[output_viz, output_summary, pdf_output, error_output]
    )

if __name__ == "__main__":
    # For Google Colab, use share=True to get a public link
    if IN_COLAB:
        print("🔬 Launching AFM Biomaterial Analyzer in Google Colab...")
        print("📊 This will generate a public URL you can share!")
        demo.launch(share=True, debug=True)
    else:
        demo.launch()

/tmp/ipython-input-797813390.py:513: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="AFM Biomaterial Compatibility Analyzer", theme=gr.themes.Soft()) as demo:


🔬 Launching AFM Biomaterial Analyzer in Google Colab...
📊 This will generate a public URL you can share!
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://13889073f109823cfc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://13889073f109823cfc.gradio.live
